In [4]:
import json 
from datasets import Dataset, load_dataset

!wget https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl
with open('./total_kor_multiturn_counsel_bot.jsonl', 
          'r', 
          encoding='utf-8') as file:
    original_jsonl_data = [json.loads(line) for line in file]
print(len(original_jsonl_data))
original_jsonl_data[0]


--2025-01-21 06:07:07--  https://raw.githubusercontent.com/MrBananaHuman/CounselGPT/main/total_kor_multiturn_counsel_bot.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30560672 (29M) [application/octet-stream]
Saving to: ‘total_kor_multiturn_counsel_bot.jsonl’

total_kor_multiturn 100%[===================>]  29.14M  87.2MB/s    in 0.3s    

2025-01-21 06:07:07 (87.2 MB/s) - ‘total_kor_multiturn_counsel_bot.jsonl’ saved [30560672/30560672]

8731


[{'speaker': '상담사', 'utterance': '안녕하세요. 상담사입니다. 무엇이 불편하시나요?'},
 {'speaker': '내담자', 'utterance': '내가 약간 중2병 같은 걸 증상을 보이고 있어요.'},
 {'speaker': '상담사', 'utterance': '중2병 증상이라니, 어떤 증상이신 건가요?'},
 {'speaker': '내담자',
  'utterance': '그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런 거예요.'},
 {'speaker': '상담사',
  'utterance': '그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?'},
 {'speaker': '내담자',
  'utterance': '막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.'},
 {'speaker': '상담사',
  'utterance': '그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교를 가면서 나아질 것 같았는데, 오히려 더 심해진 것 같다고 하셨죠. 그 원인이 무엇인가요?'},
 {'speaker': '내담자', 'utterance': '그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.'},
 {'speaker': '상담사', 'utterance': '대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?'},
 {'speaker': '내담자',
  'utterance': '학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이 만나서 그런 것 같아요.'},
 {'speaker': '상담사',
  'utterance': '즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음을 풀기도 하신다고 하셨는데, 언제부터 그 습관이 생겨난 건가요?'},
 {'speaker': '내담자',
  'utterance': '좋은 질문이에요. 좀

In [5]:
speaker_dict = {'내담자': 'user', '상담사': 'assistant'}

def preprocess_conversation(messages):
    # speaker를 role로 변환
    converted_messages = [{'role': speaker_dict[m['speaker']], 'content': m['utterance']} for m in messages]
    
    # assistant로 시작하는 경우 첫 메시지 제거
    if converted_messages and converted_messages[0]['role'] == 'assistant':
        converted_messages = converted_messages[1:]
    
    # user로 끝나는 경우 마지막 메시지들 제거
    while converted_messages and converted_messages[-1]['role'] == 'user':
        converted_messages = converted_messages[:-1]
    
    # 연속된 동일 역할의 메시지 병합
    converted_messages = merge_consecutive_messages(converted_messages)
    
    # 대화가 비어있거나 홀수 개의 메시지만 남은 경우 처리
    if not converted_messages or len(converted_messages) % 2 != 0:
        return []
    
    return converted_messages

def merge_consecutive_messages(messages):
    if not messages:
        return []
    
    merged = []
    current_role = messages[0]['role']
    current_content = messages[0]['content']
    
    for message in messages[1:]:
        if message['role'] == current_role:
            current_content += " " + message['content']
        else:
            merged.append({'role': current_role, 'content': current_content})
            current_role = message['role']
            current_content = message['content']
    
    merged.append({'role': current_role, 'content': current_content})
    return merged


def transform_to_new_format(original_data):
    transformed_data = []
    for conversation in original_data:
        processed_conversation = preprocess_conversation(conversation)
        if processed_conversation:
            transformed_data.append(processed_conversation)
    return transformed_data

result = transform_to_new_format(original_jsonl_data)
with open("./train_dataset.jsonl", "w", encoding="utf-8") as file:
    for conversation in result:
        json_obj = {"messages": conversation}
        json.dump(json_obj, file, ensure_ascii=False)
        file.write("\n") 
dataset = load_dataset("json", data_files="./train_dataset.jsonl")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 8731
    })
})

In [7]:
dataset['train'][0]

{'messages': [{'role': 'user', 'content': '내가 약간 중2병 같은 걸 증상을 보이고 있어요.'},
  {'role': 'assistant', 'content': '중2병 증상이라니, 어떤 증상이신 건가요?'},
  {'role': 'user',
   'content': '그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런 거예요.'},
  {'role': 'assistant',
   'content': '그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?'},
  {'role': 'user', 'content': '막 공부 안하고 이것저것 들먹이고 하고 싶은 게 너무 많아서 그런 거 같아요.'},
  {'role': 'assistant',
   'content': '그런 것도 어쩔 수 없이 찾아오는 시기가 있으니까 무리하지 않도록 해야겠죠. 대학교를 가면서 나아질 것 같았는데, 오히려 더 심해진 것 같다고 하셨죠. 그 원인이 무엇인가요?'},
  {'role': 'user', 'content': '그걸 제가 잘 몰라서 그런 것 같아요. 그냥 더 심해졌다고 느꼈어요.'},
  {'role': 'assistant', 'content': '대학교 생활이 신나고 재밌으신 건 어떤 점이 있나요?'},
  {'role': 'user',
   'content': '학과가 정말 좋아서 즐겁게 수업을 듣고 있어요. 학우들도 좋고 괜찮은 친구들도 많이 만나서 그런 것 같아요.'},
  {'role': 'assistant',
   'content': '즐거운 일도 많이 있으면서 고민거리도 있는 것 같군요. 가사나 소설을 쓰시면서 마음을 풀기도 하신다고 하셨는데, 언제부터 그 습관이 생겨난 건가요?'},
  {'role': 'user',
   'content': '좋은 질문이에요. 좀 자세히 말씀드릴게요. 학교에서 어려운 일이 있었는데, 그 때부터 가사나 소설 같은 것들을 쓰면서 마